In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 

In [5]:
y_hat = tf.constant(36, name = 'y_hat')
y = tf.constant(39, name = 'y')

loss = tf.Variable((y - y_hat)**2, name = 'loss')
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loss))

9


In [6]:
a = tf.constant(2)
b = tf.constant(10)

c = tf.multiply(a,b)

with tf.Session() as sess:
    print(sess.run(c))

20


In [7]:
x = tf.placeholder(tf.int64, name = 'x')
with tf.Session() as sess:
    print(sess.run(2*x, feed_dict = {x: 3}))

6


In [16]:
def linear_function(X, W, b): 
    Y = tf.matmul(W, X) + b
    with tf.Session() as sess:
        result = sess.run(Y)
    return result

In [17]:
X = tf.constant(np.random.randn(3,1), name = "X") #X to be a random tensor of shape (3,1)
W = tf.constant(np.random.randn(4,3), name = "W")
b = tf.constant(np.random.randn(4,1), name = "b")
linear_function(X, W, b)

array([[-1.13150763],
       [-2.97164943],
       [-0.82163161],
       [-1.94465327]])

In [18]:
def sigmoid(z):
    x = tf.placeholder(tf.float32, name = 'x')
    sigmoid = tf.sigmoid(x)
    with tf.Session() as sess:
        result = sess.run(sigmoid, feed_dict = {x: z})
    return result

In [20]:
sigmoid(12)

0.9999938

`tf.nn.sigmoid_cross_entropy_with_logits(logits = ...,  labels = ...)` -- have to include 'logits ='

$$- \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log \sigma(z^{[2](i)}) + (1-y^{(i)})\log (1-\sigma(z^{[2](i)})\large )\small\tag{2}$$



In [27]:
# logits = z, labels = y(true labels)
def calculateCost(logits, labels):
    z = tf.placeholder(tf.float32, name = 'z')
    y = tf.placeholder(tf.float32, name = 'y')
    
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits = z, labels = y)
    with tf.Session() as sess:
        cost = sess.run(cost, feed_dict = {z: logits, y: labels})
    return cost


cost = [0.79813886 0.91301525 0.40318605 0.34115386]


In [ ]:
logits = np.array([0.2,0.4,0.7,0.9])

cost = calculateCost(logits, np.array([0,0,1,1]))
print ("cost = " + str(cost)) 

In [29]:
# C -- number of classes, the depth of the one hot dimension
# example j had a label i. Then entry (i,j) will be 1. 

def one_hot_matrix(labels, C):
    C = tf.constant(C, name = 'C')
    one_hot_matrix = tf.one_hot(labels, C, axis = 0)
    with tf.Session() as sess:
        one_hot = sess.run(one_hot_matrix)
        
    return one_hot

In [32]:
labels = np.array([1,2,5,0,2,1]) #5 - none of the labels is true 
one_hot_matrix(labels, C = 4)

array([[0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

In [33]:
# Creates an array of ones of dimension shape
def ones(shape):
    ones = tf.ones(shape)
    with tf.Session() as sess:
        ones = sess.run(ones)

    return ones

In [34]:
ones(3)

array([1., 1., 1.], dtype=float32)

In [ ]:
def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, shape = (n_x, None))
    Y = tf.placeholder(tf.float32, shape = (n_y, None))
    
    return X, Y

In [ ]:
X, Y = create_placeholders(12288, 6)

In [ ]:
# Xavier initialization for weights and zero initialization for bias
def initialize_parameters():
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 6 lines of code)
    W1 = tf.get_variable("W1", [25,12288], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
    W2 = tf.get_variable("W2", [12,25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
    W3 = tf.get_variable("W3", [6,12], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [6,1], initializer = tf.zeros_initializer())
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [ ]:
# Implements the forward propagation for the model: 
# LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX


def forward_propagation(X, parameters):
    """
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.matmul(W1, X) + b1                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.matmul(W2, A1) + b2                                              # Z2 = np.dot(W2, A1) + b2
    A2 = tf.nn.relu(Z2)                                               # A2 = relu(Z2)
    Z3 = tf.matmul(W3, A2) + b3                                             # Z3 = np.dot(W3, A2) + b3
    
    return Z3

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(12288, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    print("Z3 = " + str(Z3))

In [ ]:
def compute_cost(Z3, Y):
    # Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    # Y -- "true" labels vector placeholder, same shape as Z3

    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    
    return cost

X_train -- training set, of shape (input size = 12288, number of training examples = 1080)  
Y_train -- test set, of shape (output size = 6, number of training examples = 1080)  
X_test -- training set, of shape (input size = 12288, number of training examples = 120)  
Y_test -- test set, of shape (output size = 6, number of test examples = 120)  

parameters -- parameters learnt by the model. They can then be used to predict.  

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):

    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    X, Y = create_placeholders(n_x, n_y)
    parameters = initialize_parameters()

    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0.                      
            num_minibatches = int(m / minibatch_size) 
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})                
                epoch_cost += minibatch_cost / num_minibatches

            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per fives)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

In [ ]:
parameters = model(X_train, Y_train, X_test, Y_test)